In [1]:
import torch
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
import torch.nn.functional as F

/opt/conda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Charger le modèle GPT-2 LARGE et le tokenizer
model_name = "gpt2-large"
tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)  # Mode évaluation

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model  = model.to(device)

In [4]:
# Préparer le prompt
prompt = "The Space Needle is in"
# 'inputs' est un dictionnaire contenant les représentations numériques du prompt
inputs = tokenizer(prompt, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)


In [5]:
outputs.keys()


odict_keys(['logits', 'past_key_values'])

In [6]:
# Récupérer les logits pour le token suivant (dernier token de la séquence)
logits = outputs.logits  # forme: (batch_size, séquence, vocab_size)
next_token_logits = logits[0, -1, :]

In [7]:
logits.shape # 1 ligne = 1 prompte, 6 colonnes = 6 token, et 50 257 mots de vocabulaires possible (token possible)

torch.Size([1, 6, 50257])

In [8]:
# Conversion des logits en probabilités
probabilities = F.softmax(next_token_logits, dim=0)

In [9]:
# Affichage des 10 tokens les plus probables
top_k = 10
top_probs, top_indices = torch.topk(probabilities, top_k)
top_tokens = [tokenizer.decode([idx]) for idx in top_indices]



In [10]:
print("Top 10 des prédictions:")
for token, prob in zip(top_tokens, top_probs):
    print(f"{token.strip()}: {prob.item():.4f}")

Top 10 des prédictions:
the: 0.2416
Seattle: 0.2009
a: 0.0793
no: 0.0188
fact: 0.0181
danger: 0.0170
need: 0.0168
good: 0.0165
an: 0.0132
downtown: 0.0110
